# 方程式の解

方程式とは未知数を含む等式のことである。方程式を解くとは、その等式を満たすような未知数を求めるということである。方程式にはさまざまな種類のものがある。例えば、

$$
x^2 - x - 6 = 0 ,
\tag{1}
$$

のような（未知数 $x$ に対する）代数方程式であったり、

$$
e^x = 3 x ,
\tag{2}
$$

のような超越方程式であったり、あるいは、

$$
\begin{cases}
2 x + 3 y = 6 , \\
\phantom{2} x + \phantom{3} y = 1 ,
\end{cases}
\tag{3}
$$

のような（未知数 $x$ と $y$ に対する）連立二元一次方程式であったりする。今回は、式 (1) や (2) のように未知数の数が一つである方程式を考えることにしよう[<sup id="cite_ref-1">[1]</sup>](#cite_note-1)。その未知数は $x$ とし、右辺が $0$ でない場合はそれを左辺に移項することができるので、一般に、

$$
f(x) = 0 ,
\tag{4}
$$

と書くことができる。式 (1) のような二次方程式であれば解の公式を使うことができるが、五次以上の代数方程式を（代数的に）解くことのできる一般の公式は存在しないことが知られている。また、式 (2) のような超越方程式は、一般に、解析的に解くことができない。よって、式 (4) を満たすような未知数 $x$ を数値的に求めよう、というのがここでの目的である。

方程式によっては、解が複数個ある場合もあるが、今回はそのうちの**一つだけ**をどうにかして求めることができればよい、ということにしよう。一般に、方程式の複数の解をすべて求めるということは、非常に難しい問題である。

**注意**：実際の研究では、よく整備されたライブラリがすでにあれば、それを利用すべきである。今回の場合で言うと、例えば [SciPy](https://www.scipy.org/scipylib/) というライブラリに [scipy.optimize.root_scalar](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.root_scalar.html) という関数が存在する。ただし、今回習うような基本的なアルゴリズムを知っておくと、ライブラリの中で何をやっているかを理解しやすくなり、想像していた解が得られないなど困難にぶつかった場合に原因を特定しやすくなるだろう。

## 関数の概形を描く

「彼（敵）を知り己を知れば百戦殆うからず」という孫子の言葉がある。まずは、今回の「敵」である式 (4) の $f(x)$ のような関数の概形を知るために、グラフを簡単に描く関数を定義しよう。

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np


def make_plot(f, x1, x2):
    # 関数f(x)に対し、x1からx2までの範囲のグラフを表示する関数。
    xpoints = np.linspace(x1, x2, 200)
    ypoints = np.vectorize(f)(xpoints)
    fig, ax = plt.subplots()
    ax.plot(xpoints, ypoints)
    ax.axhline(color='r')  # x軸（y=0）
    ax.grid()
    plt.show()

上で定義した `make_plot` 関数の使用例として、$x^2 - x - 6 = 0$ の左辺を $-4 \le x \le 4$ の範囲で描いてみよう。

In [ ]:
def f(x):
    return x ** 2 - x - 6


make_plot(f, -4, 4)

$x^2 - x - 6 = 0$ の解が $x = -2$ 付近と $x = 3$ 付近にあることが分かるだろう。（実際に $x = -2$ と $x = 3$ が解である。）

## ラムダ式

上の例では、関数 `f` を定義して、それを `make_plot` 関数に引数として渡している。これでも特に問題はないが、わざわざ（中身はたった一行の短い `return` 文であるような）関数を定義し、それに `f` という名前を与えている。後で関数 `f` を何回も再利用するのでもなければ、これは少々仰々しいとも思える。

Python では、このようなちょっとした関数を表すための[ラムダ式](https://docs.python.org/ja/3/tutorial/controlflow.html?highlight=%E3%83%A9%E3%83%A0%E3%83%80#lambda-expressions)というものが用意されており、名前のない小さな関数を作ることができる。例えば、`lambda x: x ** 2` と書くと、引数 `x` を取り、`x ** 2` を返す関数となる。上の例を、ラムダ式で作った関数を直接 `make_plot` に渡すことで書くと、次のようになる。

In [ ]:
make_plot(lambda x: x ** 2 - x - 6, -4, 4)

## 練習問題

(1) $e^x - 3x = 0$ について、グラフを描くことによって、解がどの辺りにあるか検討せよ（解は二つある）。$e^x$については、 [math モジュール](https://docs.python.org/ja/3/library/math.html)の [exp 関数](https://docs.python.org/ja/3/library/math.html#math.exp)を使えばよい。

## ニュートン法

![ニュートン法](images/newton.png)
図1. ニュートン法

いま、関数 $f(x)$ とその導関数 $f'(x)$ が分かっているものとして、$f(x)=0$ の解を数値的に求めることを考えよう。ある $x$ の値 $x_0$ を適当に取り、曲線 $f(x)$ 上の点 $(x_0, f(x_0))$ を通る接線を引く（図 1）。ただし、$f'(x) \ne 0$ と仮定する。その接線と $x$ 軸との交点の $x$ 座標を $x_1$ とすると、$x_1$ は $x_0$ よりも $f(x) = 0$ を満たす $x$ の値に（少なくとも図 1の場合には）近づいていることが分かるだろう。$x_1$ の値は次のように求めることができる：

$$
x_1 = x_0 - \frac{f(x_0)}{f'(x_0)} .
\tag{5}
$$

次に、曲線 $f(x)$ 上の点 $(x_1, f(x_1))$ を通る接線を引き、その接線と $x$ 軸との交点の $x$ 座標を $x_2$ とする。このとき、$x_2$ は $x_1$ よりも解に近い値となっている。この操作を繰り返すことにすると、次の漸化式を得る：

$$
x_{n+1} = x_n - \frac{f(x_n)}{f'(x_n)} .
\tag{6}
$$

式 (6) の反復によって近似解を求める方法をニュートン法（Newton's method）と呼ぶ。

In [ ]:
def newton(f, fprime, x0, maxiter=100, verbose=False):
    # 方程式f(x)=0の解を、x0から出発してニュートン法で求めて返す関数。
    # fprime(x)は微分を返すとする。
    #
    # x0: 初期値
    # maxiter: ニュートン法の反復回数
    # verbose: Trueなら途中の収束状況を表示
    x = x0
    
    for i in range(maxiter):
        if verbose:
            print(f'it={i}, x={x}')
        
        # 漸化式
        x = x - f(x) / fprime(x)
    
    return x

上で定義した `newton` 関数を $x^2 - x - 6 = 0$ に対して用いてみよう。

In [ ]:
newton(lambda x: x ** 2 - x - 6, lambda x: 2 * x - 1, 4)

複数の解がある場合、初期値を変えると、収束する先の解も変わる。

In [ ]:
newton(lambda x: x ** 2 - x - 6, lambda x: 2 * x - 1, -4)

ニュートン法は、初期値 $x_0$ が十分に解に近ければ、非常に速く収束することが知られている。また、一変数実関数の求解だけでなく、複素関数や多変数関数の求解へと拡張することも可能である。しかし、導関数 $f'(x)$ が必要であり[<sup id="cite_ref-2">[2]</sup>](#cite_note-2)、また $f'(x) = 0$ となる点があると、ゼロ除算によるエラーや、うまく収束しないこともある。

## 練習問題

(2) `newton` 関数は `verbose=True` をオプション引数に与えると、収束の状況を逐一表示する。上の例（$x^2 - x - 6 = 0$）に対して、初期値 $x_0$ をいろいろと変えながら、収束の様子を確かめよ。

(3) ニュートン法を用いて、$\sqrt{2}$ の値を求めよ。（つまり $x^2 = 2$ を考える。）

(4) $e^x = 3 x$ の解を求めよ。

(5) $x + 3 \sin(x) = 0$ のグラフを描いて概形を確認せよ。初期値を (a) $x_0 = 0.1$、(b) $x_0 = 3$ と取って、ニュートン法を実行し、何が起こるか確かめよ。

## 二分法

![二分法](images/bisect.png)
図 2. 二分法

いま、ある $x$ の値 $x_1$ と $x_2$ に対し、$f(x_1)$ と $f(x_2)$ が同符号でないことが分かっているとする。すると、関数 $f(x)$ が区間 $[x_1, x_2]$ で連続であれば、中間値の定理により、この区間内に $f(x) = 0$ の解が少なくとも一つは存在する。ここで $x_1$ と $x_2$ の中点：

$$
x_\text{mid} = \frac{x_1 + x_2}{2} ,
\tag{7}
$$

を考える。もし $f(x_\text{mid})$ が $f(x_1)$ と同符号であれば、区間 $[x_\text{mid}, x_2]$ に少なくとも一つの解がある。もし $f(x_\text{mid})$ が $f(x_2)$ と同符号であれば、区間 $[x_1, x_\text{mid}]$ に少なくとも一つの解がある。このようにして、解の存在する区間を半分にすることができる。これを繰り返すと、いくらでも解の存在する区間を小さくすることができる（二分法（bisection method））。

In [ ]:
def bisect(f, x1, x2, maxiter=100, verbose=False):
    # 方程式f(x)=0の、x1からx2の間にある一つの解を二分探索によって返す関数。
    # ただし、f(x1)とf(x2)は同符号ではないとする。
    #
    # maxiter: 二分探索の分割数
    # verbose: Trueなら途中の収束状況を表示
    
    # まずはy1 = f(x1)とy2 = f(x2)を計算する。
    y1 = f(x1)
    y2 = f(x2)    

    # f(x1)とf(x2)は同符号で合ってはならない。
    if y1 * y2 > 0:
        print('エラー: f(x1)とf(x2)が同符号')
        return  # 注意：エラーの時は値を何も返さない。

    # x1<=x<=x2の間の解を二分探索で探す。
    for i in range(maxiter):
        if verbose:
            print(f'it={i}, x1={x1}, x2={x2}')
    
        # 中間での関数の値を求める。
        xmid = (x1 + x2) / 2
        ymid = f(xmid)
        
        # 求めた値がy1とy2のどちらと同符号かを調べ、
        # その結果によって探索範囲を狭める。
        if ymid * y1 > 0:
            x1 = xmid
            y1 = ymid
        else:
            x2 = xmid
            y2 = ymid
    
    # x1とx2の中間値を近似解として返す。
    return (x1 + x2) / 2

上で定義した `bisect` 関数を $x^2 - x - 6 = 0$ に対して用いてみよう。注意として、求めたい解を挟んで、$x_1$ と $x_2$ を $f(x_1)$ と $f(x_2)$ が異符号となるように選ぶ必要がある。

In [ ]:
bisect(lambda x: x ** 2 - x - 6, -5, 0)

In [ ]:
bisect(lambda x: x ** 2 - x - 6, 0, 5)

## 練習問題

(6) 二分法を使って (a) $e^x = 3 x$、(b) $x + 3 \sin(x) = 0$ の解を求めよ。

(7) 「彼（敵）を知り己を知れば百戦殆うからず」と言うように、戦いに勝つには「己」を知る必要がある。ニュートン法と二分法のそれぞれの長所と短所をまとめよ。（必要であれば調べよ。）

## 演習課題

上の例題で定義された `newton` と `bisect` について、**そのまま変更せずに使うのなら**解答セルにそのコピーを書かなくてもよいこととする。以下の問題ではニュートン法と二分法のどちらを用いてもよい。

(1) 次の方程式の、$x=0$ **以外**の解のうち一つを求めよ。

$$
\sin(x) = \frac{x}{2} ,
\tag{8}
$$

In [ ]:
# 課題解答13.1  <-- 提出する際に、この行を必ず含めること。

from math import cos, sin




(2) 次の方程式の、$x = 0$ **以外**の解を求めよ。この式は、プランクの法則からウィーンの変位則を導くときに現れる[<sup id="cite_ref-3">[3]</sup>](#cite_note-3)。

$$
(x - 5)  e^{x} + 5 = 0 .
\tag{9}
$$

In [ ]:
# 課題解答13.2  <-- 提出する際に、この行を必ず含めること。

from math import exp




(3) 次の方程式の解を、$a = 1$ の場合について求めよ。ただし、平方根の中が正であるという条件によって $x$ の範囲が制限されることに注意せよ。また、$x = 0$ では分母が 0 となり、ゼロ除算によるエラーが起こることにも注意が必要である。この式は、量子力学で有限の高さの井戸型ポテンシャルでのエネルギー固有値を計算するときに現れる[<sup id="cite_ref-4">[4]</sup>](#cite_note-4)。

$$
\tan(x) = \frac{\sqrt{a-x^2}}{x} .
\tag{10}
$$

In [ ]:
# 課題解答13.3  <-- 提出する際に、この行を必ず含めること。

from math import cos, sqrt, tan




## 脚注

<span id="cite_note-1">1.</span> [^](#cite_ref-1)
式 (3) のような連立一次方程式の求解は、線形代数学における一大テーマであり、一般の場合（未知数の数が 100 個とか 1000 個とかだったり、そもそも線形独立でない式が含まれていたりしたとき）に、いかに数値的に安定して高速に解を求めるかは厄介な問題であるが、ここでは取り扱わない。

<span id="cite_note-2">2.</span> [^](#cite_ref-2)
導関数が簡単に求まらない場合のアルゴリズムには、差分によって導関数を近似する割線法（secant method）や、その一般化である準ニュートン法（quasi-Newton method）が知られている。

<span id="cite_note-3">3.</span> [^](#cite_ref-3)
黒体放射の強度のピークを求める。プランクの公式
$$
B(\lambda,T) d\lambda d\Omega = \frac{2 h c^2}{\lambda^5} \frac{1}{\exp\left(\frac{hc}{\lambda k_B T}\right) - 1}  d\lambda d\Omega,
$$
より $B(\lambda,T)$ を $\lambda$ について偏微分したとき、これが $0$ となる必要がある。$x=hc/(\lambda k_B T)$ とおくと式 (9) が得られる。

<span id="cite_note-4">4.</span> [^](#cite_ref-4)
時間に依存しない一次元シュレディンガー方程式、

$$
\left[ - \frac{\hbar^2}{2m} \frac{d^2}{dx^2} + V(x) \right] \psi(x) = E \psi(x) ,
$$

において、いま（原点に対して対称な）有限の高さの井戸型ポテンシャル、

$$
V(x) =
\begin{cases}
  0 , & |x| \le L , \\
  V_0 , & |x| > L ,
\end{cases}
$$

に対する束縛状態（$E < V_0$）を考える。ポテンシャルが偶関数なので、波動関数 $\psi(x)$ は偶関数か奇関数である。よって $x \ge 0$ のみを考えれば十分であり、$\psi(x)$ が偶関数だと仮定すると、無限遠で $\psi \to 0$ であるのと、$x=L$ での対数微分（$\psi'/\psi$）の連続性より、

$$
k \tan(kL) = \kappa , \qquad
k = \frac{\sqrt{2 m E}}{\hbar} , \qquad
\kappa = \frac{\sqrt{2 m (V_0 - E)}}{\hbar} = \sqrt{\frac{2 m V_0}{\hbar^2} - k^2} ,
$$

という条件が得られる（奇関数の場合は、$- k \cot(kL) = \kappa$ である）。ここで $\xi = kL$ とおくと、

$$
\tan(\xi) = \frac{\sqrt{\frac{2mV_0L^2}{\hbar^2} - \xi^2}}{\xi} ,
$$

となり、これを満たす $\xi$ が分かれば、エネルギー固有値を求めることができる。（教科書によっては、作図による解の求め方が書いてある。）